In [1]:
import inspect
from bs4 import BeautifulSoup as bs
import re

# import the file from wich you want to create the documentation
# it must be a .py file, if you have a Notebbok, you can copy and paste the code in a .py file and to some clean up
# !!! the .py file must not contain any import from library you don't want in the documentation
# from instance import numpy as np, importe pandas as pd must be removed from .py file
import Tube_kundt as module



#### Get the function of a module

In [ ]:
dir(module) # list of all functions, built in function and imported module

In [ ]:
# retrieve a function
obj = getattr(module,'Class1')
# filter the objects
if inspect.isfunction(obj):
    print('is a function !')
elif inspect.isclass(obj):
    print('is a class !')
else:
    print('not a function ! ')

In [ ]:
obj.__name__, obj.__doc__ # usefull information about the object

In [ ]:
# Retrieve all the object of the module
all_obj = inspect.getmembers(module)
# Filter only the function
functions = [m for m in all_obj if inspect.isfunction(m[1])]
functions

In [ ]:
obj_fc = getattr(module,'rms')
obj_cl = getattr(module,'Class1')

obj_fc.__doc__,obj_cl.__doc__


In [ ]:
n = obj_cl(1)
for i in inspect.getmembers(n):
    if i[0][0] != '_':
        print(i)

#not possible to have the attributs of a class and of their methods
# plus, it would be to chaotic
# the main attributes should be summarized in the __init__ description as returns + description: the initialized attributes are described in returns section

for i in dir(obj_cl): 
    if i[0] != '_':
        print(i)
        

        

## Create template

In [7]:
def description_object_function(obj):
    """Extract the information from an object that is a function

    Args:
        obj (object): an object

    Returns:
        desc_dict (dict) : dictionnary that stores the attributs of the object
    """
    
    # divide the sring into: description, arguments, returns
    #  desc_dict = {'name':'','type':'','input_arguments':'','summary':'','arguments':{},'returns':{},'children':{}}

    desc = obj.__doc__
    # split into lines and removes the white spaces before and after the string
    # remove the empty string
    desc_split= desc.splitlines()
    desc_split = [m.strip() for m in desc_split] 
    desc_split = list(filter(lambda x: x != "", desc_split))

    desc_dict = {'name':'','type':'','input_arguments':'','summary':'','arguments':{},'returns':{},'children':{}}

    # retrieve the arguments of the function
    desc_dict['input_arguments'] = list(obj.__code__.co_varnames) 
    desc_dict['name'] = obj.__name__
    desc_dict['type'] = 'function'

    section = 'summary'
    i = 0
    j= 0
    for sub_string in desc_split:
        
        if sub_string == 'Args:':
            section = 'arguments'
            continue
        elif sub_string == 'Returns:':
            section = 'returns'
            continue

        if section == 'arguments':
            
            # extract argument informations
            arg_detail = {'name':'','type':'','description':''}
            arg_split_1 = sub_string.split(':')
            arg_detail['description'] = ':'.join(arg_split_1[1:])
            arg_split_2 = arg_split_1[0].split('(')
            arg_detail['name'] = arg_split_2[0].strip()
            arg_detail['type'] = arg_split_2[1][:-1].strip()

            desc_dict[section][i] = arg_detail
            i+=1
            
        elif section == 'summary':
            desc_dict[section] += sub_string

        elif section == 'returns':
            return_split = sub_string.split(':')
            return_detail = {'type':'','description':''}
            return_detail['description'] = return_split[1].strip()
            return_detail['type'] = return_split[0].strip()
            desc_dict[section][j] = return_detail
            j+=1

    return desc_dict




def description_object_class(obj):
    """Extract the information from an object that is a class

    Args:
        obj (object): an object

    Returns:
        desc_dict (dict) : dictionnary that stores the attributs of the object
    """
    # divide the sring into: description, arguments, returns
    #  desc_dict = {'name':'','type':'','input_arguments':'','summary':'','arguments':{},'returns':{},'children':{}}

    desc = obj.__doc__
    # split into lines and removes the white spaces before and after the string
    # remove the empty string
    desc_split= desc.splitlines()
    desc_split = [m.strip() for m in desc_split] 
    desc_split = list(filter(lambda x: x != "", desc_split))

    desc_dict = {'name':'','type':'','input_arguments':'','summary':'','arguments':{},'returns':{},'children':{}}

    # retrieve the arguments of the function
    print(obj.__name__)
    try: desc_dict['input_arguments'] = list(obj.__code__.co_varnames) 
    except: pass
    desc_dict['name'] = obj.__name__
    desc_dict['type'] = 'function'

    section = 'summary'
    i = 0
    j= 0
    for sub_string in desc_split:
        
        if sub_string == 'Args:':
            section = 'arguments'
            continue
        elif sub_string == 'Returns:':
            section = 'returns'
            continue

        if section == 'arguments':
            
            # extract argument informations
            arg_detail = {'name':'','type':'','description':''}
            arg_split_1 = sub_string.split(':')
            arg_detail['description'] = ':'.join(arg_split_1[1:])
            arg_split_2 = arg_split_1[0].split('(')
            arg_detail['name'] = arg_split_2[0].strip()
            arg_detail['type'] = arg_split_2[1][:-1].strip()

            desc_dict[section][i] = arg_detail
            i+=1
            
        elif section == 'summary':
            desc_dict[section] += sub_string

        elif section == 'returns':
            return_split = sub_string.split(':')
            return_detail = {'type':'','description':''}
            return_detail['description'] = return_split[1].strip()
            return_detail['type'] = return_split[0].strip()
            desc_dict[section][j] = return_detail
            j+=1

    return desc_dict


In [8]:
def has_children(obj_all):   
    """Test wether or not an object has method on its own. The function returns True if there is any.

    Args:
        obj_all (object): an object

    Returns:
        bool (bool) : True if the object has children.
    """
    
    for obj in inspect.getmembers(obj_all): # [('av_mean', <function test_import_doc.av_mean(amplitudes, N=10)>),
        if obj[0][0] != '_':
            return True
        
    return False


In [10]:
def get_desc_children(module):
    """Extract the descriptions of all the functions/classes of a module.

    Args:
        module (module): module

    Returns:
        list_f (list): list of dictionnary that store the description of each function or class.
    """
    
    list_f = []
    for obj in inspect.getmembers(module): # [('av_mean', <function test_import_doc.av_mean(amplitudes, N=10)>),
        if obj[0][0] != '_':
            
            print(obj[1].__name__)
            desc_dict = {'name':'','type':'','input_arguments':'','summary':'','arguments':{},'returns':{},'children':{}}

            if inspect.isfunction(obj[1]) :
                desc_dict = description_object_function(obj[1])
            elif inspect.isclass(obj[1]):
                desc_dict = description_object_class(obj[1])

            if has_children(obj[1]):
                desc_dict['children'] = get_desc_children(obj[1])

            list_f += [ desc_dict ]

    return list_f  

In [12]:
def html_arg(arguments):
    if arguments != {}:
        string = '<div class="arg_return_band">Arguments:</div><ul>'
        for arg in list(arguments.values()):
            string+=f'<li>{arg["name"]}({arg["type"]}): {arg["description"]}</li>'
            
        string += '</ul>'
        return string
    return ''

def html_ret(returns):
    if returns != {}:
        string = '<div class="arg_return_band">Returns:</div><ul>'
        for ret in list(returns.values()):
            string += f'<li>{ret["type"]}: {ret["description"]}</li>'
        string += '</ul>'
        return string
    return ''

def html_child(children):
    if children != {}:
        string = '<div class="arg_return_band">Children:</div><ul>'
        for child_name in [child['name'] for child in children]:
            string += f'<li><a href="#{child_name}" class="a_child">{child_name}</a></li>'
        string += '</ul>'
        return string
    return ''

def html_block_function(description):
    # html_fc = f'<h3  class="func_name" >' + description["name"] + f'<a id="{description["name"]}"></a>'
    html_fc = f'<h3  class="func_name" >' + description["name"] + f'<a name="{description["name"]}"></a>'
    html_fc += '</h3> <div class="func_info"><p class="p_summary"> '+ description["summary"]+' </p>'
    html_fc += html_arg(description['arguments'])
    html_fc += html_ret(description['returns'])
    html_fc += html_child(description['children'])
    html_fc += '</div>'

    return html_fc


In [13]:
def html_link_function(description_object_dict):
    return f'<li><a href="#{description_object_dict["name"]}" class="a_sum">{description_object_dict["name"]}</a></li>'

def get_html_children(obj_desc_all):
    html_link = '<ul>'
    html_fc = ''

    for obj_desc in obj_desc_all: # [('av_mean', <function test_import_doc.av_mean(amplitudes, N=10)>),
        print(obj_desc)
        html_fc += html_block_function(obj_desc) # add the list of child in block function
        html_link += html_link_function(obj_desc)

        if obj_desc['children'] !={}:
            # add list of link
            html_link_child,html_fc_child = get_html_children(obj_desc['children'])
            html_link +=  html_link_child
            html_fc +=  html_fc_child

    html_link += '</ul>'

    return html_link,html_fc
    

In [19]:
def create_html(module,file_name='Kundt_doc.html',template_name = 'template.html'):
    general_json = get_desc_children(module)
    html_link,html_fc = get_html_children(general_json) 

    # open the template to be modified
    html = open( 'template.html')
    
    # sarse HTML file with Beautiful Soup
    soup = bs(html, 'html.parser')
    
    # store the text to be altered
    old_text = soup.find("div", {"class": "split_summary"})
    old_text2 = soup.find("div", {"class": "split_main"})
    
    # replace the stored text with the new text
    new_text = old_text.find(text=re.compile(
        '[LINK]')).replace_with(bs(html_link, 'html.parser'))
    
    new_text2 = old_text2.find(text=re.compile(
        '[FUNCTION]')).replace_with(bs(html_fc, 'html.parser'))
    
    # create a new html file
    with open("Kundt_doc.html", "w", encoding = 'utf-8') as f_output:
        f_output.write(soup.prettify())
        f_output.close()

C:\Users\mtlsa\AppData\Local\Temp\ipykernel_27492\4133165615.py:23: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  new_text = old_text.find(text=re.compile(
C:\Users\mtlsa\AppData\Local\Temp\ipykernel_27492\4133165615.py:26: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  new_text2 = old_text2.find(text=re.compile(
